<a href="https://colab.research.google.com/github/asis012/bert/blob/master/Multitasking(RTE%26%26QNLI).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
from snorkel.mtl.model import MultitaskModel
from snorkel.mtl.trainer import Trainer


In [5]:
cd ..

/


In [6]:
cd snorkel-superglue-master

/snorkel-superglue-master


In [0]:

# !pip install git+git://github.com/HazyResearch/snorkel.git@snorkel-superglue


In [0]:
import os, sys
import pandas as pd
from pathlib import Path
import json
if not "cwd" in globals():
    cwd = Path(os.getcwd())


In [0]:
TASK_NAME = "QNLI"
BERT_MODEL = "bert-large-cased"
split = "valid"

dataloader_config = {
    "batch_size": 5,
    "data_dir":( "/snorkel-superglue-master/Data"),
    "splits": ["train", "valid"],
    "max_data_samples" : 1200,
}
trainer_config = {
    "lr": 2e-5,
    "optimizer": "adamax",
    "n_epochs": 5,
    "conter_unit": "epochs",
    "evaluation_freq": 0.5,
}

In [0]:
from dataloaders import get_jsonl_path
from superglue_parsers.wic import get_rows


In [99]:
from dataloaders import get_dataloaders

dataloaders = get_dataloaders(
    task_name=TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14
15
16
17
18
19
20
21
22
23
24
25
26
27
28
29
30
31
32
33
34
35
36
37
38
39
40
41
42
43
44
45
46
47
48
49
50
51
52
53
54
55
56
57
58
59
60
61
62
63
64
65
66
67
68
69
70
71
72
73
74
75
76
77
78
79
80
81
82
83
84
85
86
87
88
89
90
91
92
93
94
95
96
97
98
99
100
101
102
103
104
105
106
107
108
109
110
111
112
113
114
115
116
117
118
119
120
121
122
123
124
125
126
127
128
129
130
131
132
133
134
135
136
137
138
139
140
141
142
143
144
145
146
147
148
149
150
151
152
153
154
155
156
157
158
159
160
161
162
163
164
165
166
167
168
169
170
171
172
173
174
175
176
177
178
179
180
181
182
183
184
185
186
187
188
189
190
191
192
193
194
195
196
197
198
199
200
201
202
203
204
205
206
207
208
209
210
211
212
213
214
215
216
217
218
219
220
221
222
223
224
225
226
227
228
229
230
231
232
233
234
235
236
237
238
239
240
241
242
243
244
245
246
247
248
249
250
251
252
253
254
255
256
257
258
259
260
261
262
263
264
265
266
267
268
269
270
271
272
273
274
275
276
27

In [0]:
from superglue_tasks import task_funcs

task = task_funcs[TASK_NAME](BERT_MODEL)

In [0]:
from snorkel.mtl.model import MultitaskModel
from snorkel.mtl.trainer import Trainer

model = MultitaskModel(tasks=[task], device=0, dataparallel=True)

In [13]:
# trainer = Trainer(**trainer_config)
# trainer.train_model(model, dataloaders)


Epoch 4:: 100%|██████████| 240/240 [02:31<00:00,  7.43s/it, QNLI/SuperGLUE/train/loss=0.103, model/all/train/loss=0.103, model/all/train/lr=2e-5, QNLI/SuperGLUE/valid/accuracy=0.787]


In [64]:
model

MultitaskModel(name=MultitaskModel)

In [0]:
model.save("./QNLI.pth")

In [0]:
model.load("/snorkel-superglue-master/QNLI.pth")

In [0]:
/snorkel-superglue-master/QNLI.pth

In [65]:
results = model.predict(dataloaders[1], return_preds=True)
golds, preds = results["golds"][TASK_NAME], results["preds"][TASK_NAME]

KeyError: ignored

In [0]:
from snorkel.slicing.sf import slicing_function

In [0]:
# @slicing_function()
# def slice_temporal_preposition(example):
#     temporal_prepositions = ["after", "before", "past"]
#     both_sentences = example.sentence1 + example.sentence2
# #     print(example.sentence1,"@@@@@@@",example.sentence2,"########",both_sentences,"!!!!!!!!!!!!!!")
#     print(any([p in both_sentences for p in temporal_prepositions]))
#     return any([p in both_sentences for p in temporal_prepositions])

# @slicing_function()
# def slice_possessive_preposition(example):
#     possessive_prepositions = ["inside of", "with", "within"]
#     both_sentences = example.sentence1 + example.sentence2
# #     return any([p in both_sentences for p in possessive_prepositions])

In [0]:
@slicing_function()
def slice_is_comparative(example):
    comparative_words = ["more", "less", "better", "worse", "bigger", "smaller"]
    both_sentences = example.sentence1 + example.sentence2
    return any([p in both_sentences for p in comparative_words])

In [0]:
@slicing_function()
def slice_is_quantification(example):
    quantification_words = ["all", "some", "none"]
    both_sentences = example.sentence1 + example.sentence2
    return any([p in both_sentences for p in quantification_words])

In [0]:
@slicing_function()
def slice_where(example):
    sentences = example.sentence1 + example.sentence2
    return "where" in sentences

@slicing_function()
def slice_who(example):
    sentences = example.sentence1 + example.sentence2
    return "who" in sentences

@slicing_function()
def slice_what(example):
    sentences = example.sentence1 + example.sentence2
    return "what" in sentences

@slicing_function()
def slice_when(example):
    sentences = example.sentence1 + example.sentence2
    return "when" in sentences

In [0]:
@slicing_function()
def slice_and(example):
    sentences = example.sentence1 + example.sentence2
    return "and" in sentences

@slicing_function()
def slice_but(example):
    sentences = example.sentence1 + example.sentence2
    return "but" in sentences

@slicing_function()
def slice_or(example):
    sentences = example.sentence1 + example.sentence2
    return "or" in sentences

In [0]:
@slicing_function()
def slice_multiple_articles(example):
    sentences = example.sentence1 + example.sentence2
    multiple_indefinite = sum([int(x == "a") for x in sentences.split()]) > 1 \
        or sum([int(x == "an") for x in sentences.split()]) > 1
    multiple_definite = sum([int(x == "the") for x in sentences.split()]) > 1
    return multiple_indefinite or multiple_definite

In [0]:
@slicing_function()
def slice_short_hypothesis(example, thresh=5):
    return len(example.sentence2.split()) < thresh

@slicing_function()
def slice_long_hypothesis(example, thresh=15):
    return len(example.sentence2.split()) > thresh

@slicing_function()
def slice_short_premise(example, thresh=10):
    return len(example.sentence1.split()) < thresh

@slicing_function()
def slice_long_premise(example, thresh=100):
    return len(example.sentence1.split()) > thresh

In [0]:
slicing_functions = [
    slice_temporal_preposition,
    slice_possessive_preposition,
    slice_is_comparative,
    slice_is_quantification,
    slice_where,
    slice_who,
    slice_what,
    slice_when,
    slice_and,
    slice_or,
    slice_but,
    slice_multiple_articles,
    slice_short_hypothesis,
    slice_long_hypothesis,
    slice_short_premise,
    slice_long_premise,
    
]

In [0]:
slice_names = [sf.name for sf in slicing_functions]





In [0]:
from snorkel.slicing.utils import add_slice_labels, convert_to_slice_tasks

# make slices tasks

slice_tasks = convert_to_slice_tasks(task, slice_names)

In [29]:
from snorkel.slicing.apply import PandasSFApplier
from snorkel.slicing.utils import add_slice_labels
from utils import task_dataset_to_dataframe

applier = PandasSFApplier(slicing_functions)
# print(applier)
# add slice labels
for dl in dataloaders:
#     print(dl)
    df = task_dataset_to_dataframe(dl.dataset)
#     print(df)
    S_matrix = applier.apply(df)
#     print("ok",S_matrix,"ok")
#     print("(((((())))))")
    
    # updates dataloaders in place
    add_slice_labels(dl, task, S_matrix, slice_names)
    
    

 29%|██▉       | 350/1200 [00:00<00:00, 1714.37it/s]

True
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
False
False
False
True
False
False
False
False
False
True
False
False
True
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
Fal

 61%|██████    | 733/1200 [00:00<00:00, 1811.96it/s]

False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
True
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
True
False
False
False
False
False
Fals

 93%|█████████▎| 1121/1200 [00:00<00:00, 1875.22it/s]

...


False
False
True
False
True
True
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
True
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
F

 73%|███████▎  | 877/1200 [00:00<00:00, 1757.41it/s]

False
False
False
False
False
False
False
True
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
True
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False

100%|██████████| 1200/1200 [00:00<00:00, 1777.11it/s]


False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
True
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False
True
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
False
True
False
False

In [0]:
slice_model = MultitaskModel(tasks=slice_tasks, device=0, dataparallel=False)



In [0]:
slice_model.load_state_dict(model.collect_state_dict())

In [45]:
trainer = Trainer(**trainer_config)
trainer.train_model(slice_model, dataloaders)

Epoch 0::   0%|          | 0/240 [00:00<?, ?it/s]/usr/local/lib/python3.6/dist-packages/snorkel/slicing/modules/slice_combiner.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for slice_ind_name in slice_ind_op_names
/usr/local/lib/python3.6/dist-packages/snorkel/slicing/modules/slice_combiner.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for slice_pred_name in slice_pred_op_names
Epoch 0:: 100%|██████████| 240/240 [02:35<00:00,  7.43s/it, QNLI/SuperGLUE/train/loss=0.0919, QNLI_slice:slice_temporal_preposition_ind/SuperGLUE/train/loss=0.273, QNLI_slice:slice_temporal_preposition_pred/SuperGLUE/train/loss=0.154, QNLI_slice:slice_possessive_preposition_ind/SuperGLUE/train/loss=0.5, QNLI_slice:slice_possessive_preposition_pred/SuperGLUE/train/loss=0.103, QNLI_slice:slice_is_comparative_ind/SuperGLUE/train/loss=0.247, QNLI_slic

In [51]:
results = slice_model.predict(dataloaders[1], return_preds=True)
golds, preds = results["golds"][TASK_NAME], results["preds"][TASK_NAME]

/usr/local/lib/python3.6/dist-packages/snorkel/slicing/modules/slice_combiner.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for slice_ind_name in slice_ind_op_names
/usr/local/lib/python3.6/dist-packages/snorkel/slicing/modules/slice_combiner.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for slice_pred_name in slice_pred_op_names


In [100]:
slice_model.score(dataloaders[1])

/usr/local/lib/python3.6/dist-packages/snorkel/slicing/modules/slice_combiner.py:40: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for slice_ind_name in slice_ind_op_names
/usr/local/lib/python3.6/dist-packages/snorkel/slicing/modules/slice_combiner.py:47: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  for slice_pred_name in slice_pred_op_names


{'QNLI/SuperGLUE/valid/accuracy': 0.7783333333333333}

In [0]:
slice_model.save("./slice_qnli.pth")

In [69]:
!unzip RTE.zip

Archive:  RTE.zip
   creating: RTE/
  inflating: RTE/train.jsonl         
   creating: __MACOSX/
   creating: __MACOSX/RTE/
  inflating: __MACOSX/RTE/._train.jsonl  
  inflating: RTE/test.jsonl          
  inflating: __MACOSX/RTE/._test.jsonl  
  inflating: RTE/val.jsonl           
  inflating: __MACOSX/RTE/._val.jsonl  
  inflating: __MACOSX/._RTE          


In [54]:
print(golds)
print(preds)
a = golds != preds
print(sum(a))

[1 2 2 ... 2 1 2]
[1 2 2 ... 2 1 1]
266


In [0]:

if not "cwd" in globals():
    cwd = Path(os.getcwd())
sys.path.insert(0, str(cwd.parents[0]))
from superglue_parsers.wic import get_rows
TASK_NAME = "RTE"
BERT_MODEL = "bert-large-cased"

dataloader_config = {
    "batch_size": 5,
    "data_dir": os.environ.get("SUPERGLUEDATA", os.path.join(str(cwd.parents[0]), "/snorkel-superglue-master/Data")),
    "splits": ["train", "valid"],
}

trainer_config = {
    "lr": 2e-5,
    "optimizer": "adamax",
    "n_epochs": 5,
    "conter_unit": "epochs",
    "evaluation_freq": 0.5,

}

In [111]:
dataloaders

In [104]:
dataloaders

In [0]:
from dataloaders import get_jsonl_path
from superglue_parsers.wic import get_rows

task_name = "RTE"
split = "valid"
max_data_samples = None # max examples to include in dataset

jsonl_path = get_jsonl_path(data_dir, task_name, split)


In [116]:
from dataloaders import get_dataloaders

dataloaders = get_dataloaders(
    task_name=TASK_NAME,
    tokenizer_name=BERT_MODEL,
    **dataloader_config
)

/snorkel-superglue-master/Data/RTE/train.jsonl
/snorkel-superglue-master/Data/RTE/val.jsonl


In [117]:
from superglue_tasks import task_funcs

task = task_funcs[TASK_NAME](BERT_MODEL)

100%|██████████| 1242874899/1242874899 [00:34<00:00, 36402355.18B/s]


In [0]:
from snorkel.mtl.model import MultitaskModel
from snorkel.mtl.trainer import Trainer

vanilla_model = MultitaskModel(tasks=[task], device=0, dataparallel=False)

In [0]:
vanilla_model.load("/snorkel-superglue-master/Data/slice_qnli.pth")

In [120]:
vanilla_model.score(dataloaders[1])

{'RTE/SuperGLUE/valid/accuracy': 0.51985559566787}

In [0]:
results = vanilla_model.predict(dataloaders[1], return_preds=True)
golds, preds = results["golds"][TASK_NAME], results["preds"][TASK_NAME]

In [123]:
print(golds)
print(preds)
a = golds != preds
print(sum(a))
print(len(a))

[2 1 2 2 1 1 1 1 1 1 1 2 2 1 1 2 2 1 1 2 1 1 1 1 2 2 2 1 2 2 2 1 2 2 1 2 2
 1 2 2 1 2 1 1 1 2 1 2 2 2 2 1 2 1 1 2 1 2 2 1 1 2 2 1 2 2 1 1 1 1 1 1 2 1
 1 2 1 2 2 1 2 1 2 2 2 1 1 2 1 2 1 2 1 1 2 2 2 1 1 1 1 1 1 1 1 1 1 2 1 1 1
 1 1 2 2 1 1 2 2 2 2 2 2 1 2 2 2 2 1 1 2 1 1 2 1 1 2 1 2 2 2 1 2 1 2 1 1 2
 1 1 2 2 1 1 1 1 2 1 2 2 1 1 2 2 1 1 2 2 2 2 1 1 2 1 1 1 1 1 2 2 1 1 2 2 1
 1 1 1 1 1 2 1 2 1 2 1 2 1 2 2 1 1 1 2 1 2 2 2 2 2 1 1 1 2 1 2 1 2 2 1 1 1
 2 2 1 2 1 1 2 1 2 1 2 2 1 1 2 1 2 2 1 1 2 2 2 2 2 1 2 2 2 1 2 1 2 1 1 1 2
 1 1 2 1 1 2 2 1 2 1 2 2 2 1 1 1 1 2]
[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 2 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 2 1 1 2 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 2 1 1 1 1 1 1 2 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1